<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/biagiominio/telerilevamento_2021/blob/main/MoD_r.ipynb#scrollTo=S32LTQVpIUKH">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/biagiominio/telerilevamento_2021/blob/main/MoD_r.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

In [ ]:
#@title **Install rgee dependencies**
system('sudo add-apt-repository ppa:ubuntugis/ubuntugis-unstable')
system('sudo apt-get update')
system('sudo apt-get install libudunits2-dev libgdal-dev libgeos-dev libproj-dev')
system('sudo apt-get install libprotobuf-dev protobuf-compiler libv8-dev libjq-dev')
install.packages('sf')
install.packages('mapview')
install.packages('cptcity')
install.packages('geojsonio')
remotes::install_github("r-spatial/rgee")

In [ ]:
library(rgee)
ee_Initialize(display = TRUE)

In [ ]:
library(sf)
library(cptcity)
library(mapview)

In [ ]:
ita <- ee$FeatureCollection('USDOS/LSIB_SIMPLE/2017')$filter(ee$Filter$eq('country_na', 'Italy'))      

In [ ]:
region <- ita$geometry()$bounds()    

Recupera il dataset MODIS Terra Vegetation Indices 16-Day Global 250m come un ee.ImageCollection e seleziona la banda EVI.

In [ ]:
col <- ee$ImageCollection('MODIS/006/MOD13Q1')$select('EVI')

Raggruppa le immagini per data composta

In [ ]:
col <- col$map(function(img) {
  doy <- ee$Date(img$get('system:time_start'))$getRelative('day', 'year')
  img$set('doy', doy)
})
distinctDOY <- col$filterDate('2010-01-01', '2011-01-01')

In [ ]:
filter <- ee$Filter$equals(leftField = 'doy', rightField = 'doy');

In [ ]:
join <- ee$Join$saveAll('doy_matches')
joinCol <- ee$ImageCollection(join$apply(distinctDOY, col, filter))

In [ ]:
comp <- joinCol$map(function(img) {
  doyCol = ee$ImageCollection$fromImages(
    img$get('doy_matches')
  )
  doyCol$reduce(ee$Reducer$median())
})


In [ ]:
visParams = list(
  min = 0.0,
  max = 9000.0,
  bands = "EVI_median",
  palette = c(
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
    )
)

In [ ]:
rgbVis <- comp$map(function(img) {
  do.call(img$visualize, visParams) %>% 
    ee$Image$clip(ita)
})

In [ ]:
gifParams <- list(
  region = region,
  dimensions = 500,
  crs = 'EPSG:3857',
  framesPerSecond = 5
)

In [ ]:
print(rgbVis$getVideoThumbURL(gifParams))
browseURL(rgbVis$getVideoThumbURL(gifParams))

In [ ]:
system("sudo apt-get install libmagick++-dev", intern = TRUE)

In [ ]:
install.packages("magick")

In [ ]:
data_modis <- c('Jan', 'Jan',
                'Feb', 'Feb',
                'Mar', 'Mar',
                'Apr', 'Apr',
                'May', 'May',
                'Jun', 'Jun',
                'Jul', 'Jul',
                'Aug', 'Aug',
                'Sept', 'Sept',
                'Oct',
                'Nov', 'Nov',
                'Dec', 'Dec')

In [ ]:
data_modis <- c('January','January',
                'February','February',
                'March','March',
                'April','April',
                'May','May',
                'June','June',
                'June','June',
                'August','August',
                'September','September',
                'October',
                'November','November',
                'December','December')

In [ ]:
library(magick)
animation <- ee_utils_gif_creator(rgbVis, gifParams, mode = "wb")
animation %>% 
  ee_utils_gif_annotate(
    text = "EVI: MODIS/006/MOD13Q1",
    size = 13, color = "white",
    location = "+10+475"
  ) %>% 
  ee_utils_gif_annotate(
    text = "_____________________", 
    size = 30, 
    location = "+280+30",
    color = "black", 
    font = "arial",
    kerning = 3,
    weight = 400,
    boxcolor = "#000000"
  ) %>% 
  ee_utils_gif_annotate(
    text = data_modis, 
    size = 25, 
    location = "+280+30",
    color = "white", 
    font = "arial",
    kerning = 0,
    weight = 400,
    boxcolor = "#000000"
  ) #-> animation_wtxt  

#ee_utils_gif_save(animation_wtxt, path = "raster_as_ee.gif")